# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


/Users/jg/anaconda/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [3]:
# frequency table for prestige and whether or not someone was admitted
# shows how undergrad ranking affects admissions
pd.crosstab(df.admit, df.prestige)

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


In [4]:
admit_by_percent = pd.crosstab( df.admit, df.prestige ).apply( lambda p: p/p.sum(), axis = 0 )
admit_by_percent

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,0.459016,0.641892,0.768595,0.820896
1,0.540984,0.358108,0.231405,0.179104


In [5]:
admit_by_percent = pd.DataFrame( admit_by_percent.unstack() ).reset_index()
admit_by_percent

,prestige,admit,0
0,1.0,0,0.459016
1,1.0,1,0.540984
2,2.0,0,0.641892
3,2.0,1,0.358108
4,3.0,0,0.768595
5,3.0,1,0.231405
6,4.0,0,0.820896
7,4.0,1,0.179104


In [6]:
admit_by_percent.columns = ['prestige', 'admit', 'admit_percent']

In [7]:
admit_by_percent

,prestige,admit,admit_percent
0,1.0,0,0.459016
1,1.0,1,0.540984
2,2.0,0,0.641892
3,2.0,1,0.358108
4,3.0,0,0.768595
5,3.0,1,0.231405
6,4.0,0,0.820896
7,4.0,1,0.179104


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [8]:
prestige_dummies = pd.get_dummies(df.prestige, 'prestige')
prestige_dummies.head()

,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: n-1

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [9]:
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np

In [10]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(prestige_dummies)
print handCalc.head()

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             0             1             0
1      1  660.0  3.67             0             0             1             0
2      1  800.0  4.00             1             0             0             0
3      1  640.0  3.19             0             0             0             1
4      0  520.0  2.93             0             0             0             1


In [11]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
pd.crosstab(handCalc['admit'], handCalc['prestige_1.0'])

prestige_1.0,0,1
admit,,
0,243,28
1,93,33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [12]:
rank_1 = pd.crosstab(handCalc['admit'], handCalc['prestige_1.0']).apply(lambda r: r/r.sum(), axis=0)
rank_1.columns = ['Rank_1', 'Rank_less_than_1']
rank_1

,Rank_1,Rank_less_than_1
admit,,
0,0.723214,0.459016
1,0.276786,0.540984


Now, it is evident that people with a undergrad ranking of 1 have much higher chances (~90%) of getting into grad school compared to only 10% without a ranking of 1.

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [13]:
also_ranked = df[df.prestige !=1]
also_ranked.prestige.value_counts()

2.0    148
3.0    121
4.0     67
Name: prestige, dtype: int64

#### 3.3 Calculate the odds ratio

In [14]:
rank_less_than_1 = pd.crosstab(also_ranked['admit'], also_ranked['prestige']).apply(lambda r: r/r.sum(), axis=0)
# rank_less_than_1.columns = ['rank_2.0', 'rank_3.0', 'rank4.0']
rank_less_than_1

prestige,2.0,3.0,4.0
admit,,,
0,0.641892,0.768595,0.820896
1,0.358108,0.231405,0.179104


#### 3.4 Write this finding in a sentenance: 

Answer:   
Your likelihood of admission decreases with each step down in rank

#### 3.5 Print the cross tab for prestige_4

In [15]:
prestige_4odds = pd.crosstab(handCalc['admit'], handCalc['prestige_4.0'], rownames=['admit'])
prestige_4odds

prestige_4.0,0,1
admit,,
0,216,55
1,114,12


#### 3.6 Calculate the OR 

In [16]:
prestige_4odds.apply( lambda p4: p4/p4.sum(), axis=0)

prestige_4.0,0,1
admit,,
0,0.654545,0.820896
1,0.345455,0.179104


#### 3.7 Write this finding in a sentence

Answer:  
The probability graduate acceptance is higher for students who attending top ranked undergraduate school

## Part 4. Analysis

In [17]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(handCalc.ix[:, 'prestige_2.0':])
print data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             1             0
1      1  660.0  3.67             0             1             0
2      1  800.0  4.00             0             0             0
3      1  640.0  3.19             0             0             1
4      0  520.0  2.93             0             0             1


/Users/jg/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  This is separate from the ipykernel package so we can avoid doing imports until


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [18]:
# manually add the intercept
data['intercept'] = 1.0

In [19]:
data.columns

Index([u'admit', u'gre', u'gpa', u'prestige_2.0', u'prestige_3.0',
       u'prestige_4.0', u'intercept'],
      dtype='object')

#### 4.1 Set the covariates to a variable called train_cols

In [20]:
train_cols = data.columns[1:]
train_cols

Index([u'gre', u'gpa', u'prestige_2.0', u'prestige_3.0', u'prestige_4.0',
       u'intercept'],
      dtype='object')

#### 4.2 Fit the model

In [21]:
logit = sm.Logit(data['admit'], data[train_cols])
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [22]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Fri, 15 Dec 2017   Pseudo R-squ.:                 0.08166
Time:                        13:41:16   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa              0.7793      0.333      2.344      0.019       0.128       1.431
prestige_2.0    -0.6801      0.317     -2.146      0.032      -1.301      -0.059
prestige_3.0    -1.3387      0.345     -3.882      0.000      -2.015      -0.663
prestige_4.0    -1.5534      0.417     -3.721      0.000      -2.372      -0.735
intercept       -3.8769      1.142     -3.393      0.001      -6.116      -1.638
================================================================================
"""

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [23]:
np.exp(result.params)

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64

In [24]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
np.exp(conf)

,2.5%,97.5%,OR
gre,1.000074,1.004372,1.002221
gpa,1.136120,4.183113,2.180027
prestige_2.0,0.272168,0.942767,0.506548
prestige_3.0,0.133377,0.515419,0.262192
prestige_4.0,0.093329,0.479411,0.211525
intercept,0.002207,0.194440,0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer:   
gre has equal probabilistic outcomes, since the OR is > 1

There is no way to express in one number how X affects Y in terms of probability. The effect of X on the probability of Y has different values depending on the value of X.

#### 4.6 Interpret the OR of GPA

Answer: 

The OR is > 2  
- In its low range it predicts 1.13 times greater likelihood of admission
- In its high range it predicts 4.18 times greater likelihood of admission


## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [25]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [26]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian( [ gres, gpas, [1, 2, 3, 4], [1.] ] ) )

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [27]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
dummy_ranks = pd.get_dummies(combos['prestige'], prefix='prestige')

In [28]:
dummy_ranks.columns

Index([u'prestige_1.0', u'prestige_2.0', u'prestige_3.0', u'prestige_4.0'], dtype='object')

In [29]:
dummy_ranks.columns = ['prestige_1.0', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0']

In [30]:
# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2.0':])


/Users/jg/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  This is separate from the ipykernel package so we can avoid doing imports until


#### 5.2 Make predictions on the enumerated dataset

In [31]:
# make predictions on the enumerated dataset
combos['admit_pred'] = result.predict(combos[train_cols])

combos.head()

,gre,gpa,prestige,intercept,prestige_2.0,prestige_3.0,prestige_4.0,admit_pred
0,220.0,2.260000,1.0,1.0,0,0,0,0.164173
1,220.0,2.260000,2.0,1.0,1,0,0,0.090492
2,220.0,2.260000,3.0,1.0,0,1,0,0.048977
3,220.0,2.260000,4.0,1.0,0,0,1,0.039890
4,220.0,2.453333,1.0,1.0,0,0,0,0.185907


#### 5.3 Interpret findings for the last 4 observations

Answer:  
- the probability of admission gradually increases as gre and gpa increase
- the different prestige levels yield drastic probabilities of admission (particularly the most/least prestigious  schools)

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.